In [1]:
#import dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
#read in merged and cleaned data
df = pd.read_csv('../modified_data/merged_cleaned.csv')
df

,FIPS,3/31/21,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,...,Gender Equality,Hopefulness,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic
0,1001.0,6589,594.443459,55200.0,23315.0,21115.0,8422.0,1065.0,29372.0,4204.0,...,77.063492,91.163142,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952
1,1003.0,20505,1589.793007,208107.0,111945.0,78622.0,21653.0,4343.0,31203.0,14310.0,...,69.016011,82.484017,28069.0,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246
2,1005.0,2227,885.001636,25782.0,11937.0,9186.0,6597.0,918.0,18461.0,4901.0,...,69.323158,61.927181,17249.0,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105
3,1007.0,2542,622.461089,22527.0,9161.0,6840.0,2863.0,658.0,20199.0,2650.0,...,68.844059,85.258871,18988.0,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677
4,1009.0,6444,644.830460,57645.0,24222.0,20600.0,8220.0,909.0,22656.0,7861.0,...,64.585114,79.492703,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,56037.0,4022,10426.975725,44117.0,19628.0,15871.0,5237.0,1213.0,32624.0,2549.0,...,68.112161,82.403142,30945.0,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334
3074,56039.0,3609,3996.844622,23059.0,13680.0,9158.0,1619.0,210.0,53703.0,958.0,...,70.618318,84.036899,46499.0,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645
3075,56041.0,2128,2081.719807,20609.0,8972.0,7735.0,2552.0,614.0,27009.0,934.0,...,71.058477,84.089095,25636.0,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391
3076,56043.0,891,2238.672972,8129.0,3868.0,3422.0,984.0,253.0,27556.0,590.0,...,67.383553,87.485019,26325.0,76.249370,77.658224,67.412774,82.820701,78.925326,74.628788,70.050103


In [3]:
#set index to FIPS
df = df.set_index(df['FIPS'])
df= df.drop('FIPS', axis=1)

In [4]:
#rename target column
df = df.rename(columns={'3/31/21':'first_year_cases'})

In [5]:
#create variable for case% for each counties population
df['case_pct'] = df['first_year_cases']/df['E_TOTPOP']*100
df['case_pct'].head()

FIPS
1001.0    11.936594
1003.0     9.853104
1005.0     8.637809
1007.0    11.284237
1009.0    11.178767
Name: case_pct, dtype: float64

In [6]:
df['case_pct'].describe()

count    3078.000000
mean        9.412435
std         3.068056
min         0.000000
25%         7.699891
50%         9.453957
75%        11.173830
max        38.010657
Name: case_pct, dtype: float64

In [7]:
# bin and cut the case_pct column into 2 classifications
# q = df['case_pct'].quantile(.75)
q = df['case_pct'].quantile(.9)
bins = [0, q , 40]
labels = ['low','high']
df['case_class'] = pd.cut(df['case_pct'], bins, labels = labels)
df['case_class']

FIPS
1001.0      low
1003.0      low
1005.0      low
1007.0      low
1009.0      low
           ... 
56037.0     low
56039.0    high
56041.0     low
56043.0     low
56045.0     low
Name: case_class, Length: 3078, dtype: category
Categories (2, object): ['low' < 'high']

In [8]:
df['case_class'].value_counts()

low     2744
high     308
Name: case_class, dtype: int64

In [9]:
#drop unneeded columns
df = df.drop('case_pct', axis =1)
df

,first_year_cases,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,...,Hopefulness,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,case_class
FIPS,,,,,,,,,,,,,,,,,,,,,
1001.0,6589,594.443459,55200.0,23315.0,21115.0,8422.0,1065.0,29372.0,4204.0,8050.0,...,91.163142,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,low
1003.0,20505,1589.793007,208107.0,111945.0,78622.0,21653.0,4343.0,31203.0,14310.0,40665.0,...,82.484017,28069.0,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,low
1005.0,2227,885.001636,25782.0,11937.0,9186.0,6597.0,918.0,18461.0,4901.0,4634.0,...,61.927181,17249.0,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,low
1007.0,2542,622.461089,22527.0,9161.0,6840.0,2863.0,658.0,20199.0,2650.0,3661.0,...,85.258871,18988.0,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,low
1009.0,6444,644.830460,57645.0,24222.0,20600.0,8220.0,909.0,22656.0,7861.0,10233.0,...,79.492703,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037.0,4022,10426.975725,44117.0,19628.0,15871.0,5237.0,1213.0,32624.0,2549.0,4721.0,...,82.403142,30945.0,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334,low
56039.0,3609,3996.844622,23059.0,13680.0,9158.0,1619.0,210.0,53703.0,958.0,3135.0,...,84.036899,46499.0,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645,high
56041.0,2128,2081.719807,20609.0,8972.0,7735.0,2552.0,614.0,27009.0,934.0,2498.0,...,84.089095,25636.0,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391,low


In [10]:
#turn case % classifications into binary 
df = pd.get_dummies(df, columns = ['case_class'])
df

,first_year_cases,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,...,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,case_class_low,case_class_high
FIPS,,,,,,,,,,,,,,,,,,,,,
1001.0,6589,594.443459,55200.0,23315.0,21115.0,8422.0,1065.0,29372.0,4204.0,8050.0,...,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,1,0
1003.0,20505,1589.793007,208107.0,111945.0,78622.0,21653.0,4343.0,31203.0,14310.0,40665.0,...,28069.0,77.232120,80.086368,71.771566,67.272980,75.586018,66.983549,70.972246,1,0
1005.0,2227,885.001636,25782.0,11937.0,9186.0,6597.0,918.0,18461.0,4901.0,4634.0,...,17249.0,80.375206,78.783778,73.657368,76.066481,78.753019,65.170377,68.704105,1,0
1007.0,2542,622.461089,22527.0,9161.0,6840.0,2863.0,658.0,20199.0,2650.0,3661.0,...,18988.0,80.813736,77.837027,69.974652,75.136154,76.929754,69.859503,67.931677,1,0
1009.0,6444,644.830460,57645.0,24222.0,20600.0,8220.0,909.0,22656.0,7861.0,10233.0,...,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037.0,4022,10426.975725,44117.0,19628.0,15871.0,5237.0,1213.0,32624.0,2549.0,4721.0,...,30945.0,79.384759,79.347081,68.147062,73.938691,76.390464,67.420658,70.956334,1,0
56039.0,3609,3996.844622,23059.0,13680.0,9158.0,1619.0,210.0,53703.0,958.0,3135.0,...,46499.0,71.547359,80.522872,65.399695,79.598153,79.698193,70.877600,70.938645,0,1
56041.0,2128,2081.719807,20609.0,8972.0,7735.0,2552.0,614.0,27009.0,934.0,2498.0,...,25636.0,78.771570,77.859042,67.603416,69.705859,73.332067,67.404487,69.299391,1,0


In [11]:
#seperate targets and features
X = df.drop(['case_class_low','case_class_high'], axis=1).values
y=df['case_class_high'].values

In [12]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
# hidden_nodes_layer1 = 100
# hidden_nodes_layer2 = 50
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 100
# hidden_nodes_layer3 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 150)               15450     
                                                                 
 dense_1 (Dense)             (None, 100)               15100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


2022-08-05 17:40:25.655058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
73/73 [==============================] - 1s 3ms/step - loss: 546.5898 - accuracy: 0.8280
Epoch 2/100
73/73 [==============================] - 0s 2ms/step - loss: 186.2650 - accuracy: 0.8523
Epoch 3/100
73/73 [==============================] - 0s 2ms/step - loss: 372.8416 - accuracy: 0.8237
Epoch 4/100
73/73 [==============================] - 0s 2ms/step - loss: 158.3256 - accuracy: 0.8484
Epoch 5/100
73/73 [==============================] - 0s 3ms/step - loss: 101.0638 - accuracy: 0.8527
Epoch 6/100
73/73 [==============================] - 0s 2ms/step - loss: 78.7409 - accuracy: 0.8713
Epoch 7/100
73/73 [==============================] - 0s 4ms/step - loss: 71.8243 - accuracy: 0.8674
Epoch 8/100
73/73 [==============================] - 0s 3ms/step - loss: 243.2597 - accuracy: 0.8618
Epoch 9/100
73/73 [==============================] - 0s 4ms/step - loss: 211.1709 - accuracy: 0.8540
Epoch 10/100
73/73 [==============================] - 0s 2ms/step - loss: 40.8884 - accuracy:

73/73 [==============================] - 0s 3ms/step - loss: 0.1845 - accuracy: 0.9723
Epoch 83/100
73/73 [==============================] - 0s 3ms/step - loss: 0.2065 - accuracy: 0.9731
Epoch 84/100
73/73 [==============================] - 0s 2ms/step - loss: 0.3816 - accuracy: 0.9649
Epoch 85/100
73/73 [==============================] - 0s 2ms/step - loss: 4.0769 - accuracy: 0.9432
Epoch 86/100
73/73 [==============================] - 0s 2ms/step - loss: 9.1612 - accuracy: 0.8869
Epoch 87/100
73/73 [==============================] - 0s 2ms/step - loss: 10.8154 - accuracy: 0.8843
Epoch 88/100
73/73 [==============================] - 0s 2ms/step - loss: 12.8644 - accuracy: 0.8791
Epoch 89/100
73/73 [==============================] - 0s 2ms/step - loss: 10.4153 - accuracy: 0.8722
Epoch 90/100
73/73 [==============================] - 0s 2ms/step - loss: 1.4196 - accuracy: 0.9315
Epoch 91/100
73/73 [==============================] - 0s 3ms/step - loss: 0.4371 - accuracy: 0.9493
Epoch 92/1

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

25/25 - 0s - loss: 11.3262 - accuracy: 0.8948 - 248ms/epoch - 10ms/step
Loss: 11.326242446899414, Accuracy: 0.8948051929473877


In [34]:
df.sort_values('first_year_cases', ascending=False)

,first_year_cases,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,...,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,case_class_low,case_class_high
FIPS,,,,,,,,,,,,,,,,,,,,,
6037.0,1219614,4058.191394,10098052.0,3524321.0,3306109.0,1589956.0,357178.0,32469.0,1460718.0,1299277.0,...,29301.0,77.300323,82.753382,65.146461,70.146341,77.275727,72.024049,72.669530,1,0
4013.0,524386,9199.246764,4253913.0,1719157.0,1520767.0,618496.0,117059.0,31697.0,354513.0,615277.0,...,28791.0,76.239268,80.318374,65.312232,66.222222,80.167548,74.312169,68.850154,1,0
17031.0,497885,944.934768,5223719.0,2188394.0,1963070.0,778020.0,214832.0,35575.0,477426.0,724730.0,...,32179.0,77.004584,81.550130,64.361206,58.534483,82.573392,72.267478,69.033124,1,0
48201.0,377152,1705.202063,4602523.0,1740973.0,1583486.0,739100.0,143208.0,31901.0,557077.0,452327.0,...,29850.0,76.680355,80.843722,67.992533,51.794872,76.370854,69.379159,71.939245,1,0
6065.0,297412,7206.555780,2383286.0,833602.0,718349.0,345724.0,94709.0,27142.0,280295.0,328609.0,...,24443.0,75.971437,80.457604,63.298793,64.545455,79.209656,74.371693,68.044384,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49027.0,0,6604.985026,12733.0,5049.0,4306.0,1436.0,184.0,25847.0,881.0,2045.0,...,21643.0,81.011322,76.161823,69.817675,73.631823,77.590921,70.417087,72.342978,0,0
49023.0,0,3391.677302,10948.0,3701.0,3390.0,1229.0,157.0,22498.0,477.0,1262.0,...,19194.0,81.530930,76.754223,74.848496,74.422640,79.427835,68.699461,70.416611,0,0
49021.0,0,3296.335949,49691.0,20807.0,16204.0,9645.0,1145.0,20861.0,1997.0,6072.0,...,19085.0,76.214915,78.797194,73.499569,76.791825,77.090871,69.497344,70.167864,0,0
